# Launch ComfyUI on Brev!

Welcome!

In this notebook, we have adapted ComfyUI's default Colab notebook to run on Brev. ComfyUI is a powerful GUI for Stable Diffusion models. It uses a node-based architecture that allows users to construct very complex image/video generation workflows by connecting different nodes, each representing a different function or operation!

In [ ]:
# Environment Setup

from pathlib import Path

OPTIONS = {}

WORKSPACE = 'ComfyUI'
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

In [ ]:
%%bash
if ! [ -d WORKSPACE ]; then
    echo "Starting initial setup of ComfyUI..."
    git clone https://github.com/comfyanonymous/ComfyUI
    if [ -d ComfyUI ]; then
        echo "Repository cloned successfully into 'ComfyUI'."
        cd ComfyUI
    else
        echo "Failed to clone repository."
    fi
else
    echo "Workspace directory already exists."
fi

In [ ]:
# Check and handle updates for ComfyUI
if OPTIONS['UPDATE_COMFY_UI']:
    !echo "Updating ComfyUI..."
    !git pull

!echo "Installing dependencies..."
!pip install xformers!=0.0.18 -r ComfyUI/requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
import os

# Check if we should use ComfyUI Manager
if 'USE_COMFYUI_MANAGER' in OPTIONS and OPTIONS['USE_COMFYUI_MANAGER']:
    # Change directory to custom_nodes if it exists
    %cd custom_nodes

    # Ensure executable permissions for various scripts
    if not os.path.isfile("ComfyUI-Manager/check.sh"):
        !chmod 755 ComfyUI-Manager/check.sh
    if not os.path.isfile("ComfyUI-Manager/scan.sh"):
        !chmod 755 ComfyUI-Manager/scan.sh
    if not os.path.isfile("ComfyUI-Manager/node_db/dev/scan.sh"):
        !chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
    if not os.path.isfile("ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh"):
        !chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
    if not os.path.isfile("ComfyUI-Manager/scripts/install-comfyui-venv-win.bat"):
        !chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

    # Clone the ComfyUI Manager repository if it does not exist
    if not os.path.isdir("ComfyUI-Manager"):
        print("-= Initial setup ComfyUI-Manager =-")
        !git clone https://github.com/ltdrdata/ComfyUI-Manager /home/ubuntu/verb-workspace/ComfyUI/custom_nodes

    # Change directory to ComfyUI-Manager and pull the latest changes
    %cd ComfyUI-Manager
    !git pull

    # Navigate back to the initial directory
    %cd ..

# Return to the workspace directory
%cd $WORKSPACE

# Install dependencies for custom nodes if required
if 'INSTALL_CUSTOM_NODES_DEPENDENCIES' in OPTIONS and OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
    print("-= Install custom nodes dependencies =-")
    !pip install GitPython
    !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

In [ ]:
import os
!pip install huggingface_hub

In [ ]:
!wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

In [ ]:
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

In [ ]:
import subprocess
import threading
import time
import socket

def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()

In [ ]:
!python main.py --listen 0.0.0.0 --port 8188 --dont-print-server --disable-xformers

In [ ]:
def check_server(port):
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            result = sock.connect_ex(('127.0.0.1', port))
            if result == 0:
                print("\nServer is up and running at http://129.80.69.99:{}".format(port))
                break
            time.sleep(0.5)

threading.Thread(target=check_server, daemon=True, args=(8188,)).start()